In [ ]:
# 定义IDW插值函数
def idw_interpolation(x, y, z, xi, yi, k=1000):
    tree = cKDTree(np.vstack((x, y)).T)
    distances, locations = tree.query(np.vstack((xi.flatten(), yi.flatten())).T, k=k)
    weights = 1 / distances
    weights /= weights.sum(axis=1)[:, None]
    # 使用循环来处理每个查询点
    zi = np.zeros(xi.shape).flatten()
    for i, loc in enumerate(locations):
        zi[i] = np.sum(weights[i] * z.iloc[loc])

    return zi.reshape(xi.shape)


In [ ]:
# 更改伦敦borough边界的CRS
London_boroughs = London_boroughs.to_crs(epsg=3857)

In [ ]:
# 将 DataFrame 转换为 GeoDataFrame
gdf = gpd.GeoDataFrame(
    gdf_listing, 
    geometry=gpd.points_from_xy(gdf_listing.longitude, gdf_listing.latitude)
)
gdf.crs = "EPSG:4326"  # 设置坐标参考系统为 WGS84
# 将数据从 WGS84 转换到 Web Mercator
gdf = gdf.to_crs(epsg=3857)
# 创建网格用于IDW插值
x_IDW = np.linspace(gdf.geometry.x.min(), gdf.geometry.x.max(), 960)
y_IDW = np.linspace(gdf.geometry.y.min(), gdf.geometry.y.max(), 960)
xx_IDW, yy_IDW = np.meshgrid(x_IDW, y_IDW)

# 执行 IDW 插值
zi = idw_interpolation(gdf.geometry.x, gdf.geometry.y, gdf['sum_income'], xx_IDW, yy_IDW)




In [ ]:
# 对插值后的zi进行双立方卷积（使边缘平滑过渡）
# 创建一个双立方核（3x3）
convolve_kernel_2cube = np.array([[1/16, 1/8, 1/16],
                   [1/8,  1/4, 1/8],
                   [1/16, 1/8, 1/16]])
# 创建一个5*5的卷积核（5x5）
convolve_kernel_5times = np.array([[1/273, 4/273, 7/273, 4/273, 1/273],
                                    [4/273, 16/273, 26/273, 16/273, 4/273],
                                    [7/273, 26/273, 41/273, 26/273, 7/273],
                                    [4/273, 16/273, 26/273, 16/273, 4/273],
                                    [1/273, 4/273, 7/273, 4/273, 1/273]])


# 对zi应用双立方卷积
zi_smoothed = convolve(zi, convolve_kernel_5times)

In [ ]:
# 正确地将扁平化的数组转换为 pandas Series
zi_flatten = pd.Series(zi.flatten())
print(zi_flatten.max())
print(zi_flatten.min())
print(zi_flatten.median())

zi_smoothed_flatten = pd.Series(zi_smoothed.flatten())
print(zi_smoothed_flatten.max())
print(zi_smoothed_flatten.min())
print(zi_smoothed_flatten.median())

# 创建图表和轴
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# 在第一个轴上绘制 zi_flatten 的直方图
ax1.hist(zi_flatten, bins=3000)  # 调整 bins 的数量以更好地展示数据分布
ax1.set_xlim(0, 20000)
ax1.set_title('Original Data Histogram')

# 在第二个轴上绘制 zi_smoothed_flatten 的直方图
ax2.hist(zi_smoothed_flatten, bins=3000)  # 同样的bins数量
ax2.set_xlim(0, 20000)
ax2.set_title('Smoothed Data Histogram')
plt.show()

In [ ]:
quantiles = np.linspace(0, 1, 41)
levels = np.quantile(zi.flatten(), quantiles)
standard_format_numbers = [format(num, 'f') for num in levels]
print(standard_format_numbers)

In [ ]:
# 对数值进行分隔
quantiles = np.linspace(0, 1, 21)
levels = np.quantile(zi_smoothed.flatten(), quantiles)
#levels = np.array([0,2000,2500,2700,2900,3000,3400,3737,3800,4000,4200,4500,4800,5000,7000,200000])

# 绘制地图
fig, ax = plt.subplots(figsize=(16, 16))
ax.contourf(xx_IDW, yy_IDW, zi_smoothed, levels=levels, cmap="tab20c_r", alpha=0.6)
London_boroughs.boundary.plot(ax=ax, edgecolor='brown', linewidth=2, alpha=0.4)

# 设置颜色规范化
norm = mcolors.Normalize(vmin=gdf['sum_income'].min(), vmax=gdf['sum_income'].max())

# 绘制点，根据sum_income列进行上色
# 这里假设gdf具有'geometry'列，包含点的位置
#scatter = ax.scatter(gdf.geometry.x, gdf.geometry.y, c=gdf['sum_income'], edgecolors=None, s=2, cmap='viridis', alpha=0.2, norm=norm)

# 添加颜色条
#plt.colorbar(scatter, ax=ax, label='Income')

#gdf.plot(ax=ax, color='black', markersize=2, alpha=0.25)               # 绘制点
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)         #添加OSM底图

plt.show()